# 🪐Project Overview🪐 


See:

* [GITHUB REPO](https://github.com/cmsc-vcu/cmsc508-fa2023-prj-astronomy_database_group53)
* [PITCH VIDEO](https://clipchamp.com/watch/5yb3Z2ORm1g?utm_source=share&utm_medium=social&utm_campaign=watch)
* [DESIGN VIDEO](https://vcu.mediaspace.kaltura.com/media/video1242889568/1_u95qqk0j)


### Problem Description

<span style="color:#a074c1;">Problem Domain</span>

: Astronomy, a captivating blend of science and human curiosity, explores the vast expanse of celestial objects, the mysteries of space, and the entirety of our physical universe. This field of study has a rich and storied history that dates back to the earliest civilizations, reflecting our innate desire to comprehend the world that surrounds us.

: In the 3rd century BCE, Greek astronomers like Galileo Galilei introduced groundbreaking methods that substantiated the heliocentric model, reshaping our understanding of the universe. 


<span style="color:#a074c1;">Need</span>

: Everything in the sky is constantly moving, and, for those who are interested in astronomy, a tool to look up locations of celestial objects would be useful. Centralizing lots of the information already available into a user-friendly database could make astronomy more accessible, and hobbies which connect you to your surroundings as well as build community are great for your mental health.

<span style="color:#a074c1;">Context, Scope, and Perspective</span>

: This database could be helpful for more than just avid astronomers. For them, we want to give information about where stars, planets, and constellations can be seen in telescopes. For more casual fans, we want to include information about events that can be seen with the naked eye - blue moons, eclipses, meteor showers, etc. Giving information on when and where they’re next predicted to happen could be useful for anyone who likes to stargaze.

<span style="color:#a074c1;">User roles and Use Cases</span>

: Amateur Astronomers and Stargazers:

* Usage: They use the database to plan a stargazing session, locate celestial objects, and gain insights into upcoming astronomical events.

* Use Case: Users can find information on upcoming celestial events, visible planets, stars, and constellations in the night sky. The database provides real-time information to plan stargazing sessions.

<span style="color:#a074c1;">Security and Privacy</span>

: Access Controls
* Unauthorized users should not be able to access or manipulate the database.

: Data Encryption
* Data transmitted over the network, especially sensitive user information such as a user's query (in case it has their location), should not be accessible by other users.

: User Authentication
* We do not plan on having a user authentication as our database will not store user's data or information. It will be purely for querying.

### Database Design

<span style="color:#a074c1;">Design Tradeoffs</span>

* We removed an idea for an astrology extension to the database. We planned for the user to input their birthdate and be able to receive information on their sun sign, any planet movements affecting their sign, and other astrology considerations. However, this would've added much more complexity to our database. So, we decided to remove this functionality.

* We also were thinking of having the user input their location, and, in return, they'd receive instructions on how to best view a celestial body or event. This would include things like the angle and direction they must look in.

* One more design consideration is how to handle Location and time. Currently, there are two different tables for coordinates and longitude and latitude coordinates. There's also Altitude and Azimuth coordinates, which, given two set of longitude and latitude and the time, or celestial coordinates and the time, you're able to get the angle and direction you'd have to look to see a celestial body/event. It's a bit of a complex calculation, and we're unsure if performance would be better having a query calculate it or creating another table.

Below is an abstract design of the database:

```{mermaid}
%%{init: {'theme':'dark'}}%%
erDiagram
    CELESTIAL-OBJECT }o--o{ EVENT : causes
    CELESTIAL-OBJECT }o--o{ CELESTIAL-OBJECT : linked_to
    OBSERVER }|--|{ CELESTIAL-OBJECT : discovers
    OBSERVER }|--|{ EVENT : discovers
    CELESTIAL-OBJECT }|--|{ LOCATION : viewed_from
    CELESTIAL-OBJECT }|--|{ CELESTIALLOCATION : has
    EVENT }|--|{ LOCATION : viewed_from
```



<span style="color:#a074c1;"> Objects and Events</span>

* Our idea is that celestial objects can cause events, although events don't always have to be directly caused by a celestial object or may have a cause we are unsure of. Many objects can cause an event, but some do not. 

<span style="color:#a074c1;">Location</span>

* Both events and ojects have locations that they can be seen from Earth. Events and objects can be seen sometimes only in certain hemispheres or regions (like Aurora Borealis). This allows the user to query the database with questions like: What location has the most meteor showers? This requires splitting the Earth into certain zones, so, when coordinates are inputted, they are classified into a certain zone.

<span style="color:#a074c1;">Observers</span>

* We also plan to add observers, where an event or object can have multiple, but an observer must discover at least one to be in the database. This way, users can query with questions like: When was the last discovery about Jupiter made? This would also require celestial objects to have other objects linked to them, such as Earth is linked to the moon.


### Our Tables - BCNF Showcase

* (you can tab through them)
* All these tables are in BCNF because their candidate keys determine all other attributes, and there are no partial dependencies.


In [ ]:
#| eval: true
#| echo: false
import os
import sys
import pandas as pd
from tabulate import tabulate
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError, ProgrammingError
import requests
from flask import Flask
sys.path.append('cmsc508-fa2023-prj-astronomy_database_group53/src') 
import json
from IPython.display import display, Markdown, HTML

app = Flask(__name__)
header = {'Authorization' :'Bearer super-secret'}

In [ ]:
#| eval: true
#| echo: false

# modify config_map to reflect credentials needed by this program
config_map = {
    'user':'CMSC508_USER',
    'password':'CMSC508_PASSWORD',
    'host':'CMSC508_HOST',
    'database':'ASTRONOMY_DB_NAME'
}
# load and store credentials
load_dotenv()
config = {}
for key in config_map.keys():
    config[key] = os.getenv(config_map[key])
flag = False
for param in config.keys():
    if config[param] is None:
        flag = True
        print(f"Missing {config_map[param]} in .env file")
#if flag:
#    sys.exit(1)

In [ ]:
#| eval: true
#| echo: false

# build a sqlalchemy engine string
engine_uri = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# create a database connection.  THIS IS THE ACTUAL CONNECTION!
try:
    cnx = create_engine(engine_uri)
except ArgumentError as e:
    print(f"create_engine: Argument Error: {e}")
    #sys.exit(1)
except NoSuchModuleError as e:
    print(f"create_engine: No Such Module Error: {e}")
    #sys.exit(1)
except Exception as e:
    print(f"create_engine: An error occurred: {e}")
    #sys.exit(1)

In [ ]:
#| echo: false
#| eval: true

# Do a quick test of the connection and trap the errors better!
try:
    databases = pd.read_sql("show databases",cnx)
except ImportError as e:
    print(f"Error: {e}")
except OperationalError as e:
    print(f"Database/SQL Error:\n{str(e)}\n")
except ProgrammingError as e:
    print(f"Programming Error:\n{str(e)}\n")
except Exception as e:
    print(f"An error occurred:\n{str(e)}\n")

::: {.panel-tabset}

## Objects

**$R_{0}$**, objects, has functional dependency

* object_id $\rightarrow$ object_name, type, description

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from objects;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Event
**$R_{1}$**, events, has functional dependencies

* event_id $\rightarrow$ event_name, date_occurred, duration, frequency

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from events;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Observer
**$R_{3}$**, observers, has functional dependency

* observer_id $\rightarrow$ first_name, last_name

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from observers;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Earth Location
**$R_{4}$**, earth_locations, has functional dependencies

* earth_location_id $\rightarrow$ quadrant, latitude, longitude, timezone, local_time, description

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from earth_locations;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Space Locations
**$R_{4}$**, space_locations, has functional dependency

* space_location_id $\rightarrow$ ra, de, description

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from space_locations;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

:::

### Our Associative Tables

::: {.panel-tabset}
## Object-Event
* Containing the event_id, the object_id, and the date. One event_id can be related to multiple object_id's, but does not need to be related to any.

- Candidate Key: `{object_id, event_id}`
- Functional Dependency: `object_id, event_id → date`

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from object_event;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Object-Location
* Containing the object_id, the location_id. Some objects can be seen from multiple locations. Not all objects can be viewed from the same location at two different times.

- Candidate Key: `{object_id, location_id}`
- Functional Dependency: `object_id, location_id → date`

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from object_location;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Object-Space Location
* Containing the space_location_id, object_id. The object has to have at least one location in space.

- Candidate Key: `{object_id, cel_location_id}`
- Functional Dependency: `object_id, cel_location_id → date`

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from object_space_location;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Object-Object
* Contains 2 object_id's. Used for cases like planets with moons, planetary conjunctions. One object can have 0 or more objects related to it.

- Candidate Key: `{object_id, related_object_id}`
- Functional Dependency: `object1_id, object2_id → date`

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from object_object;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Event-Location
* Containing the event_id, the location_id, and the date. Some events can be seen from multiple locations. Not all events can be viewed from the same location at two different times, so date is important.

- Candidate Key: `{event_id, location_id}`
- Functional Dependency: `event_id, location_id → date`

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from event_location;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Observer-Event
* Contains the observer_id, the event_id, and date. An observer must discover at least one event, and can discover multiple.

- Candidate Key: `{observer_id, event_id}`
- Functional Dependency: `observer_id, event_id → date`


In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from observer_event;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## Observer-Object
* Contains the observer_id and the object_id. An observer must discover at least one object, and can discover multiple.

- Candidate Key: `{observer_id, object_id}`
- Functional Dependency: `observer_id, object_id → date`

In [ ]:
#| echo: false
#| eval: true
sql = f"""
select * from observer_object;
"""
## Add code to list roles table here
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

::: 

### Queries

::: {.panel-tabset}

## 1
1. **Retrieve names and descriptions of objects visible from a specific location.**

   * $\pi_{object\_name, description}(\sigma_{location\_id = X}(Object-Location))$


In [ ]:
sql = f"""
     SELECT 
        object_name,
        c.location_name
    FROM
        object_location a
        INNER JOIN objects b on (a.object_id = b.object_id)
        INNER JOIN earth_locations c on (a.location_id = c.earth_location_id)
    WHERE location_id = 2
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 2
2. **Find upcoming events for a given location.**

   * $\pi_{event\_name, date}(\sigma_{location\_id = X}(Event-Location))$


In [ ]:
sql = f"""
    SELECT 
     b.event_name, 
     a.date_occurred
    FROM 
     event_location a
    JOIN events b ON a.event_id = b.event_id
    WHERE a.location_id = 5 AND a.date_occurred > CURDATE()
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 3
3. **Identify objects discovered by a specific observer.**

   * $\pi_{object\_name, first\_name,last\_name}(\sigma_{observer\_id = X}(Observer-Object))$

In [ ]:
sql = f"""
    SELECT 
     c.first_name,
     c.last_name,
     b.object_name
    FROM 
     observer_object a
    INNER JOIN objects b ON a.object_id = b.object_id
    INNER JOIN observers c on a.observer_id = c.observer_id
    WHERE a.observer_id = 8
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 4
4. **List all events related to a specific object.**

   * $\pi_{event\_name}(\sigma_{object\_id = X}(Object-Event))$


In [ ]:
sql = f"""
    SELECT 
        b.event_name
    FROM
        object_event a
        INNER JOIN events b ON (a.event_id = b.event_id)
    WHERE a.object_id = 5;
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 5
5. **Find the objects with specific characteristics (e.g., galaxies, stars).**

   * $\pi_{object\_name}(\sigma_{object\_type = "galaxy" \land "planet")}(Object)$

In [ ]:
sql = f"""
    SELECT 
        b.object_name
    FROM
        objects b
    WHERE b.type = 'galaxy' OR b.type = 'planet'
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 6
6. **Retrieve the objects observed from a given quadrant.**

   * $\pi_{object\_name}(\sigma_{quadrant = "NW"}(EarthLocation \Join Object-Location \Join Object))$

In [ ]:
sql = f"""
    SELECT 
        c.object_name
    FROM
        earth_locations a
        INNER JOIN object_location b ON (a.earth_location_id = b.location_id)
        INNER JOIN objects c ON (b.object_id = c.object_id)
    WHERE a.quadrant = 'NW';
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 7
7. **Find events on a specific date.**

   * $\pi_{event\_name}(\sigma_{date = "YYYY-MM-DD"}(Event))$

In [ ]:
sql = f"""
    SELECT 
        e.event_name
    FROM
        events e
    WHERE e.date_occurred = '2024-04-20';
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 8
8. **Get details about space locations.**

   * $\pi_{ra, de}(\sigma_{space\_location\_id = X}(SpaceLocation))$

In [ ]:
sql = f"""
    SELECT 
        a.description,
        a.ra,
        a.de
    FROM
        space_locations a
    WHERE a.space_location_id = 1 OR a.space_location_id = 2 OR a.space_location_id = 7;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 9
9. **List objects linked to other objects.**

   * $\pi_{object\_name}(\sigma_{related\_object\_id = X}(Object-Object))$


In [ ]:
sql = f"""
    SELECT 
        b.object_name
    FROM
        object_object a
        INNER JOIN objects b ON (a.object2_id = b.object_id)
    WHERE a.object1_id = 3;
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 10
10. **Find the observers who discovered a specific object.**

    * $\pi_{first\_name, last_name}(\sigma_{object\_id = X}(Observer-Object))$

In [ ]:
sql = f"""
   SELECT 
        d.first_name,
        d.last_name
    FROM
        observer_object a
        INNER JOIN observers d ON (a.observer_id = d.observer_id)
    WHERE a.object_id = 3;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 11
11. **Plan a stargazing session by retrieving the objects visible from a specific location.**

    * $\pi_{object\_name}(\sigma_{space\_location\_id = X}(Object-Location \Join SpaceLocation))$

In [ ]:
sql = f"""
   SELECT 
        c.object_name
    FROM
        object_location a
        INNER JOIN objects c ON (a.object_id = c.object_id)
    WHERE a.location_id = 2;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 12
12. **Identify the best area to observe a particular celestial event, considering location's time zone (UTC).**

    * $\pi_{event\_name, date, description, local\_time}(\sigma_{event\_id = X}(Event-Location \Join EarthLocation))$

In [ ]:
sql = f"""
    SELECT 
        b.event_name,
        b.date_occurred,
        c.location_name,
        c.local_time
    FROM
        event_location a
        INNER JOIN events b ON (a.event_id = b.event_id)
        INNER JOIN earth_locations c ON (a.location_id = c.earth_location_id)
    WHERE a.event_id = 2 AND c.local_time > '07:00:00' AND c.local_time < '23:00:00'  ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 13
13. **Explore details about the celestial objects discovered by a specific observer, including their descriptions.**

    * $\pi_{object\_name, description}(\sigma_{observer\_id = X}(Observer-Object \Join Object))$

In [ ]:
sql = f"""
    SELECT 
        d.first_name,
        d.last_name,
        b.object_name,
        b.type,
        b.description
    FROM
        observer_object a
        INNER JOIN observers d ON (a.observer_id = d.observer_id)
        INNER JOIN objects b ON (a.object_id = b.object_id)
    WHERE a.observer_id = 3;
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 14
14. **Find information about upcoming meteor showers, specifically where and when they can be observed.**

    * $\pi_{event\_name, date}(\sigma_{event\_name = "Meteor Shower"}(Event-Location))$

In [ ]:
sql = f"""
    SELECT 
        b.event_name,
        c.location_name,
        b.date_occurred,
        c.local_time
    FROM
        events b
        INNER JOIN event_location a ON (b.event_id = a.event_id)
        INNER JOIN earth_locations c ON (a.location_id = c.earth_location_id)
    WHERE b.event_name = 'meteor shower'
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 15
15. **Locate celestial objects of a particular type (e.g., planets) visible from a specific quadrant.**

    * $\pi_{object\_name}(\sigma_{object\_type = "Planet" \land quadrant = "NW"}(Object \Join Object-Location \Join EarthLocation))$

In [ ]:
sql = f"""
     SELECT 
        c.object_name
    FROM
        earth_locations a
        INNER JOIN object_location b ON (a.earth_location_id = b.location_id)
        INNER JOIN objects c ON (b.object_id = c.object_id)
    WHERE c.type = 'planet' AND a.quadrant = 'SW'
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 16
16. **Discover the events scheduled for a specific date and their locations.**

    * $\pi_{event\_name, location\_name}(\sigma_{date = "YYYY-MM-DD"}(Event-Location))$

In [ ]:
sql = f"""
   SELECT 
        b.event_name,
        c.location_name
    FROM
        event_location a
        INNER JOIN events b ON (a.event_id = b.event_id)
        INNER JOIN earth_locations c ON (a.location_id = c.earth_location_id)
    WHERE a.date_occurred = '2023-10-20';
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 17
17. **Determine the celestial objects with the highest ra**

    * $\pi_{object\_name, ra}(\sigma_{date = "YYYY-MM-DD"}(Object-Location \Join SpaceLocation))$

In [ ]:
sql = f"""
    SELECT 
        c.object_name,
        b.ra
    FROM
        object_location a
        INNER JOIN objects c ON (a.object_id = c.object_id)
        INNER JOIN space_locations b ON (a.location_id = b.space_location_id)
    WHERE a.date_occurred = '2024-03-05'
    ORDER BY b.ra DESC
    LIMIT 1
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 18
18. **Explore details about a specific space location, such as RA and Dec.**

    * $\pi_{RA, Dec}(\sigma_{location\_id = X}(EarthLocation \Join Object-Location \Join SpaceLocation))$

In [ ]:
sql = f"""
    SELECT
        b.description, 
        b.ra,
        b.de
    FROM
        earth_locations a
        INNER JOIN object_location c ON (a.earth_location_id = c.location_id)
        INNER JOIN space_locations b ON (c.location_id = b.space_location_id)
    WHERE a.earth_location_id = 2
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 19
19. **Find celestial events with unique characteristics (e.g., total solar eclipses) and their dates.**

    * $\pi_{event\_name, date}(\sigma_{event\_name = "Total Solar Eclipse"}(Event-Location))$

In [ ]:
sql = f"""
    SELECT 
        c.event_name,
        b.date_occurred
    FROM
        events c
        INNER JOIN event_location b ON(c.event_id = b.event_id)
        INNER JOIN earth_locations a ON (b.location_id = a.earth_location_id)
    WHERE c.event_name = 'super blue moon';
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

## 20
20. **Discover celestial objects with unusual relationships (e.g., moons of specific planets).**

    * $\pi_{object\_name, related\_object\_id}(\sigma_{related\_object\_id = X}(Object-Object))$

In [ ]:
sql = f"""
    SELECT 
        a.object_name,
        b.object2_id
    FROM
        objects a
        INNER JOIN object_object b ON a.object_id = b.object1_id
    WHERE b.object2_id = 8
    ;
"""

In [ ]:
#| echo: false
#| eval: true
try:
    df = pd.read_sql(sql,cnx)
    df
except Exception as e:
    message = str(e)
    print(f"An error occurred:\n\n{message}\n\nIgnoring and moving on.")
    df = pd.DataFrame()
df

:::


### API
::: {.panel-tabset}

## Astronomy API
* Our API was made in the astronomy.py file using flask
* Our client.py defines functions to easily utilize our endpoints for basic tasks involving the API

**Endpoints**
# Observers
1. get all observers
* query parameters: first_name, last_name, sort_by, order_by, per_page
2. get observer(id)
3. observer add(id, first_name, last_name)
4. remove observer(id)
5. events get all
* query parameters: name, date, duration, frequency, sort_by, order_by, per_page
6. get event(id)
7. events add
8. events remove(id)
9. get objects
* query parameters: name, type, sort_by, order_by, per_page
10. get object(id)
11. objects add
12. objects remove(id)
13. get earth_locations
* query parameters: quad, lat, long, timezone, time, sort_by, order_by, per_page
14. get earth_locations(id)
15. add earth_locations
16. remove earth_locations(id)
17. get space_locations
* query parameters: ra, de, sort_by, order_by, per_page
18. get space_locations(id)
19. add space_locations
20. remove space_locations(id)

## How to Use

1. Basic API Request:

In [ ]:
    response = requests.get('http://localhost:5000/events')

Here are the templates for the URL:

* 'http://localhost:5000/{entity}'
* 'http://localhost:5000/{entity}/{id}'
* 'http://localhost:5000/{entity}/add'
* 'http://localhost:5000/{entity}/remove/{id}'

and then instead of using get, depending on the task you use:
* requests.post to add a row
* requests.delete to delete a row

2. status code == 200
* we want to make sure our request was completed successfully

In [ ]:
    if response.status_code == 200:
        data = response.json()
        print("All Events:")
        print(json.dumps(data, indent=4))
    else:
        print(f"Request failed with status code {response.status_code}.")

:::


### Reflection

<span style="color:#a074c1;">**Astronomy Engine**</span>

* Using Astronomy Engine, we will be able to turn any location given in longitude and latitude into both Equatorial coordinates and Altitude Azimuth coordinates, as these will help provide the user with instructions to view celestial objects and events. However, we did not allocate enough time to implement this additional feature to our project. This is something we could look into implementing as our own side project.

<span style="color:#a074c1;">**Accurate Data**</span>

* Our data individually is all realistic other, however, the relations are not necessarily accurate. We just went table by table to find data. To research the links between each item is much more time intensive. For example, we did not look to see whether y planet can be from x space location and which earth location it can be.

<span style="color:#a074c1;">**User Input**</span>

* We wanted to include an element of user input where the user could input their location and/or a date and would find upcoming events in their area. However, that was out of scope of the assignment, but it would be cool to look into that on our own. It would require much more data and again the Astronomy Engine to do return useful results.